<a href="https://colab.research.google.com/github/IrfanFathan/Data-Science/blob/main/Exercise_Creating%2C_Reading_and_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2d6e02f62941d36b34a1398728e8897949d750b95576227512283c9237b964148bc3e9e49feabdaa1cd6b0f204a8163b45a9a111cf4ec54f04e0ef07f91c4b51ee8cecf377517a66c0967583ff9e46a03b6bf24da0ade83516dacab0e06cf956ddf1a03db52a8cafa765f0dabeadfeb8466962c76ac2942382376418de232f3579523275f2dd351273e572a87f9b2e581082df4f1701dc243d2d2d466bdb696cd405affd70f61b06f54b3117ba5973a6545e2b8e37b5b0a886bd17f2f3168619548f265be14f5a97e2c1b87571ebb1ba378b833bdccc395b07d813a7d288d1a608133dc6b4399429af5a3b6c99b52327ca2b9ee56d37af0cd23a4f7050686ecf,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9ac24fa8d2904b0bf57616746f6c41f8952c4fa692b02909c1ac2144baf1a997d8a91f1fb6afd79ab179b53e20da5589dcc20c11f93f731d8343593bfe202144fda7584ca7cc3ae39f8d810f47fe3f882f7adcc44bf85289e8b48d594e60dfff6a2d6661f5f0ce74326896f0deb3255e0cb36b2bf6b08a06ab9fdc65a0d42f92a02c3960a2c777ecee62b996bba2693122fb34967a0f439db62fa97b1673906e1553403d3875fdc999b286934bdcdbd2f03132899e47a927c6c494aabdcd62aff8a653c2162d381f8e1265db96ab5927319a60a62056a53bb8b36f2fa31c0f27b21cd679bce9356c87397fe6ee89e0dd77ff888708b15d29ecdbd4a6d698cf54,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D15039b85f1c71992c78ae348910db00c6e9155acb501ba260741f749159ac9099975ea2c0b2d20c4ec4d5b9857deea34bd06d406e083d14816ee634713410e1d31efad67317b7d84984c54abc9020126bfb5a956c20a4eaaab0c3e30a821b8c92a953a21889684ba68173b762bad865f3f2ec433097478b6b807666c2798acc6528d5e458711426225149c4936e8d65f2b9414b5d2e92d31e156ca18e8a470634ae95b4c29ec5fc2e6a01817802a117ab8930815f3f35ba7b0b988a454fb84fb2ee5ff64281658f9926ba47f6c8b6dcf65af46d4ec36961b2d11036241d822253a7253ed42c7c28093bb0879dfa052056effe17b643680284e5d7199143df10f,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5cf623a2bb7ed0e13f14107e810bc0a2d9efee80eebc6f2c22ccca41126c8f7b7d6130b8b844ef37466b22c226a3421534b461272bf262582271500153a6c6d0de7475601af3c89a511c68c019dfc2d1a216e880d40e5b391009b68434d2848c6f67a7af586c65f0bb98f71de1cb6ac7e73f6eb78708d242b016809e00ae0b99845cb096dd4da57eb4dd54e8abb28faea1f6f747825b347ee09d88dfb251c53adbe2c98333dc5f7cac9c74d938f2af392159680b98d2ffc7cdafa3ca53df74c66c2c619a447022c2a3445ace228e1613aae7ddfe82107d3dea0cae4a2db5e1f5081548582e1884ce7e223a0adedf00f6dad34d549e61b9398da88dcdbe9276de,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D00ae8341c5f8be9568574d9dda428ed7f50272971e6a4a65cf86751982b108e3adf6e684775ef139396c43bbaa1c3ba9c2f6a661c9d4520bb1a42e9934b0a92478c52a2947ec13969c3656c1ea6a5cd008616b6a76320521b33362e1b6458956265132aec53dd7506dc37d7f128ffaa00ae3446e6b110c978652a045ec7d5c9cad5fdcb9aa2387672c124ecf3c8fefeeed59b6d5dc007aca12e7f7599902f2eca84a62345a47364e493cf7cc45189310d8b805d9c702e9af5e604ada71599e02fa9bc6c9c6bd62557c619ec8caf3ff4dc1f36cc963b652ff43dcd374528b41871fb22afec845ff54129a53b48979ef5f4511374bcb350a587c767b7e9d847bbf,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D92f3948ec9fa7d9e119ca243d1847ff0cf87cc30b7348d6b8a8b10e671105bc1cf7f1025f1188e470219847c0e47c4c4c9245d42e0a71ec0d406b1cf05809640503ce942e68e08542150dc04be57bff04990a20018fbf65e28254cdd201a47fb042addfe16a90d6f502988db3949894039f725899382bc6e5301f6f2ec2c6dbb0f1f5600e50e7e05bcf5c68370ae4c3a2f5585b7305e70f925f27ede8184363ed0db587257525dca3eba70e3ffe11aaf99b9231f93840d05bbf83220aac39bc5e9e52780e965bdefa99ffe9125948bf5ef0d210df3988f08b4ac8dc97508c931caf3bcf924ac51cbcf842b2dde8831b61dfbf6934d5c000fd78e4d9e32625d84,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4348f676c1891caa3d86740d392e131ae7c92f69e3e736a9bc74dcd8ff65b6e3045d142a2e3ccebc361891cb59c9afd737dd7078c50419db04bc1ae3f48258457d2e8c05c637d9a2181b60e57fa1370ede8b88c28e1173ebe2ce6b0796e1c1747795a20817ddb414b18e85dc2b6cfec68536fe21f1505fb2dc51285758ad3b831aedceabce96a495d99ee109752e0f0dcd48d2dcd45e47d7a6fb4bcc4588a6ba7aa19c11dc203256fcd29d088298de44534c88436dce3c4e781f68e4705a98e5d2cce5dac174fd397d49a97e8c154dd757c7cc8962190d332c07f838a2e6e44bb09af49b01f99252454d51007c4315a68321fb1d56c619dad7b710c3029c3ca8,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5ec2360ad8a0eb13e9249447a2fc3d1e86cb5f2b3ed6700a391189ce3d25a77964b5088259b0a6832e142177278c60d791e53ae42703c9115c7723cfd58dbdc3a3f5537de6cd2e84dada93f648447a8ffc74b7976d7295de61572d63ac7f368de728ee0d612509011498dd6767f30929927954fb51ee07a3770620c74efa7033d56e120051b85550da92d9dcce6abe67e694a86143e3bd16d960e500eb862905471306e45b691d8d840909c1ad172463be36286be608a7302253bd58bbb2a173d601401fa01de11b5adc09c77fcd0619347762c43e3bfe1d3d5ac481b30dbac6e364e5aec3e332eee2f7791d9302d62f851835dad65e0767f1c479efb975f098,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2aa9df0bd2df609357d93242acfb96ed00cd373c328c5ef606c33350e7b9c80ce740f9398ab491cbd9013fd7203ebe8b92b87b6657b1ddefafd38758ebdf079f614b6cfc1b0dcbaf16910dd5d45dca80fb07de2fa93a07c9cfdc869cea30fadf371c4711b7f70287f3d939b009e0efa56cbf8306a2917d1065c0837430ba1186acf93c5e355dc60dd8a762b741c0099aa40829d2b26ef6a7e9cb0145755a2d574d46a70b9a9b5803582c2dbf3fe297134ca7210780d81ef68a0d20fc620c00940646997fdce3ecf87b0be2912a2028c1a502cdbf2d8788ba1efcac578cbc6edece78411bc4a88a494d810ca6066045ec900d7f86609c485c07125e42ee76b3a2,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T071750Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da40bb4af0ade027c6436ff0daf60f61f02d1ca6bc49d38bb2da2c5624b443bc284d807d868ba5b2fe79b5b0d596ba391be3383e80776dd166d0f7d320f3ff3ff82819bb3416140c850cf0eb32b35a81f3f395c517bf49ca9723024dc27d7bd6014cf94d115041726162cd31914b4aaf4d3e13e120c7873566ed0445a4c3ac8d564b3dbd2f0f059bf272de774a3ff2ce7f745d021ed1bd6e134a7babd0a12feb08ad6651661beaf88c18ef7ea475c5059c938bae0deb5c628633188641a83b08d34c32770643db22976fc0cbcacd1b12aad7fb8e849dac86e6868f2d8e5276d90a2832d184a9d61e11930bbb7ae21f4a1d0cd1390f649893f5633b00c80d1015d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/creating-reading-and-writing).**

---


# Introduction

The first step in most data analytics projects is reading the data file. In this exercise, you'll create Series and DataFrame objects, both by hand and by reading data files.

Run the code cell below to load libraries you will need (including code to check your answers).

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from learntools.core import binder; binder.bind(globals())
from learntools.pandas.creating_reading_and_writing import *
print("Setup complete.")

# Exercises

## 1.

In the cell below, create a DataFrame `fruits` that looks like this:

![](https://storage.googleapis.com/kaggle-media/learn/images/Ax3pp2A.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruits.
import pandas as pd
fruits = pd.DataFrame({'Apples':[30],'Bananas':[21]})

# Check your answer
q1.check()
fruits

In [ ]:
#q1.hint()
#q1.solution()

## 2.

Create a dataframe `fruit_sales` that matches the diagram below:

![](https://storage.googleapis.com/kaggle-media/learn/images/CHPn7ZF.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruit_sales.
fruit_sales =pd.DataFrame({'Apples':[35,41],'Bananas':[21,34]},index=['2017 Sales','2018 Sales'])

# Check your answer
q2.check()
fruit_sales

In [ ]:
q2.hint()
#q2.solution()

## 3.

Create a variable `ingredients` with a Series that looks like:

```
Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object
```

In [ ]:
ingredients = pd.Series(['4 cups','1 cup','2 large','1 can'],index=['Flour','Milk','Eggs','Spam'],name='Dinner')

# Check your answer
q3.check()
ingredients

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Read the following csv dataset of wine reviews into a DataFrame called `reviews`:

![](https://storage.googleapis.com/kaggle-media/learn/images/74RCZtU.png)

The filepath to the csv file is `../input/wine-reviews/winemag-data_first150k.csv`. The first few lines look like:

```
,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine[...]",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and[...]",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
```

In [ ]:
reviews =pd.read_csv('../input/wine-reviews/winemag-data_first150k.csv',index_col=0)

# Check your answer
q4.check()
reviews

In [ ]:
#q4.hint()
#q4.solution()

## 5.

Run the cell below to create and display a DataFrame called `animals`:

In [ ]:
animals = pd.DataFrame({'Cows': [12, 20], 'Goats': [22, 19]}, index=['Year 1', 'Year 2'])
animals

In the cell below, write code to save this DataFrame to disk as a csv file with the name `cows_and_goats.csv`.

In [ ]:
# Your code goes here
animals.to_csv('cows_and_goats.csv')
# Check your answer
q5.check()

In [ ]:
#q5.hint()
q5.solution()

# Keep going

Move on to learn about **[indexing, selecting and assigning](https://www.kaggle.com/residentmario/indexing-selecting-assigning)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*